# Python OSEMN process
---

---

1.- [Scrub](#scrub)

&nbsp;
    1.1.- [Pre-test](#pre)
    
&nbsp;
        1.1.1.- [Crear tabla de tiempos](#timeTable)
        
&nbsp;
        1.1.2.- [Obtener eventos en etapa de búsqueda y selección](#searchAndSelect)

2.- [Explore](#explore)
    
3.- [Model](#model)
    
4.- [Interpret](#interpret)

# 1.- Scrub 
<a id="scrub"></a>

---

## 1.1.- Pre-test
<a id="pre"></a>

In [ ]:
/Users/ianorellana/Documents/GitHub/Python/virEnv/tesis/bin/python3 -m pip install pandas

Importar pandas y numpy para manejo de dataframe

In [20]:
import pandas as pd
import numpy as np

### 1.1.1.- Crear tabla de tiempos
<a id="timeTable"></a>

Importar tabla de links visitados en la etapa de pretest

In [22]:
visitedLinksPre = pd.read_csv('Tablas generadas/Pre-test/VisitedLinks.PreTest.csv')

Revisar primeros datos de la tabla

In [37]:
visitedLinksPre.head()

,username,userId,X_id,state,url,localTimestamp,serverTimestamp
0,101BSCE120003,KnqPytrKdYvoWobR6,XnghJcJWpt9kawj6D,PageExit,/login,1488795765263,1488795765417
1,101BSCE120003,KnqPytrKdYvoWobR6,sikmeettZJj8XEhTp,PageEnter,/start,1488795765264,1488795765584
2,101BSCE120003,KnqPytrKdYvoWobR6,Dx664JM6NAwmhEpSx,PageExit,/start,1488795781045,1488795781689
3,101BSCE120003,KnqPytrKdYvoWobR6,NYhgvvR2Ky2SKwbbZ,PageEnter,/affective?stage=begin,1488795781051,1488795781699
4,101BSCE120003,KnqPytrKdYvoWobR6,ye6WFzgGgnfGb87Mp,PageExit,/affective?stage=begin,1488795820262,1488795820483


Comprobar cantidad de usuarios y filas

In [67]:
print(len(visitedLinksPre["username"].unique())," usuarios")
print(len(visitedLinksPre), " filas")

512  usuarios
56750  filas


Obtener tabla con tiempos iniciales en la etapa de búsqueda y selección

In [57]:
startTimePre = visitedLinksPre.loc[(visitedLinksPre["state"]=="PageExit") & 
                               (visitedLinksPre["url"]=="/tutorial?stage=search"),["username","serverTimestamp"]]
print(len(startTimePre["username"].unique())," usuarios")
print(len(startTimePre)," filas")

512  usuarios
515  filas


Debido a error en el sistema de captura de datos, algunos usuaris poseen duplicados en la fase tutorial.  Se eliminan todos excepto la primera ocurrencia

In [70]:
startTimePre = startTimePre.drop_duplicates(subset='username',keep='first')
print(len(startTimePre["username"].unique())," usuarios")
print(len(startTimePre)," filas")
startTimePre.columns = ["username","start"]
startTimePre.head()

512  usuarios
512  filas


,username,start
16,101BSCE120003,1488795953781
82,101BSCE120004,1488795974227
201,101BSCE120008,1488795937180
278,101BSCE120012,1488796022746
368,101BSCE120014,1488796497576


Obtener tabla con tiempos iniciales en la etapa de búsqueda y selección

In [64]:
finishTimePre = visitedLinksPre.loc[(visitedLinksPre["state"]=="PageEnter") & 
                                    (visitedLinksPre["url"]=="/collection"),["username","serverTimestamp"]]
print(len(finishTimePre["username"].unique())," usuarios")
print(len(finishTimePre)," filas")

512  usuarios
524  filas


Se observa el mismo problema que la tabla anterior, por lo que se eliminan duplicados conservando el último elemento

In [71]:
finishTimePre = finishTimePre.drop_duplicates(subset='username',keep='last')
print(len(finishTimePre["username"].unique())," usuarios")
print(len(finishTimePre)," filas")
finishTimePre.columns = ["username","finish"]
finishTimePre.head()

512  usuarios
512  filas


,username,finish
51,101BSCE120003,1488796554066
156,101BSCE120004,1488796840407
247,101BSCE120008,1488796645263
337,101BSCE120012,1488796806432
415,101BSCE120014,1488797036983


Construir una tabla donde por cada usuario se tenga el tiempo donde inició y finalizó la tarea de búsqueda y selección (join)

In [75]:
timeTablePre = pd.merge(startTimePre, finishTimePre, on='username', how='inner')
timeTablePre.head()

512


,username,start,finish
0,101BSCE120003,1488795953781,1488796554066
1,101BSCE120004,1488795974227,1488796840407
2,101BSCE120008,1488795937180,1488796645263
3,101BSCE120012,1488796022746,1488796806432
4,101BSCE120014,1488796497576,1488797036983


### 1.1.2.- Obtener eventos en etapa de búsqueda y selección
<a id="searchAndSelect"></a>

Obtener tabla de usuarios, eventos (consultas, ingreso y salida de páginas, bookmarks, entre otros), scrolls, clicks  y entradas de texto

In [94]:
usersPre = pd.read_csv('Tablas generadas/Pre-Test/Users.PreTest.csv')
print(len(usersPre), " usuarios")
usersPre.head()

512  usuarios


,child.ID,userName,T.Inicial,T.Final,Total.Time,Stay.Pages,Stay.Pag.Relv,Stay.Pag.NotRelv,Total.Cover,Doc.Relv.vist,...,Recall,F1,Score,Pos,Cal,Ask1,Ask2,Sex,Group,class
0,1109,101BSCE120003,1488795953616,1488796447580,"8,233","3,112","1,943","1,169",4,2,...,"0,667","0,572","3,333",-2,1,2,3,2,0,A
1,1121,101BSCE120004,1488795974073,1488796824341,"14,171","5,837","0,544","5,293",9,2,...,"0,667","0,333","1,667",1,3,3,2,2,0,R
2,1118,101BSCE120008,1488795936998,1488796634284,"11,621","5,521","2,739","2,782",5,3,...,1,"0,75",5,-3,3,4,2,2,0,A
3,1112,101BSCE120012,1488796022585,1488796762579,"12,333","5,655","0,947","4,708",9,2,...,"0,667","0,333","1,429",-1,0,4,3,2,0,R
4,1106,101BSCE120014,1488796497561,1488796914443,"6,948","2,051","0,587","1,464",6,3,...,1,"0,667",3,2,2,3,3,1,0,R


In [116]:
eventlogsPre = pd.read_csv('Tablas generadas/Pre-test/EventLogs.PreTest.csv')
eventlogsPre = eventlogsPre[["username","actionId","clientTimestamp","serverTimestamp","action"]]
eventlogsPre.columns = ["username","actionId","localTimestamp","serverTimestamp","action"]
print(len(eventlogsPre)," eventos")
eventlogsPre.head()

131474  eventos


,username,actionId,localTimestamp,serverTimestamp,action
0,101BSCE120003,NaN,1488795764636,1488795764636,StatusOnline
1,101BSCE120003,ixZRM67XDCcPHCdgm,1488795764777,1488795764949,Login
2,101BSCE120003,XnghJcJWpt9kawj6D,1488795765263,1488795765417,PageExit
3,101BSCE120003,sikmeettZJj8XEhTp,1488795765264,1488795765584,PageEnter
4,101BSCE120003,NaN,1488795765603,1488795765603,StatusOnline


In [110]:
mouseClicksPre = pd.read_csv("Tablas generadas/Pre-test/MouseClicks.PreTest.csv")
mouseClicksPre = mouseClicksPre[["username","X_id","localTimestamp","serverTimestamp"]]
actions = np.repeat('Click',len(mouseClicksPre))
mouseClicksPre['action'] = actions
print(len(mouseClicksPre)," clicks")
mouseClicksPre.columns = ["username","actionId","localTimestamp","serverTimestamp","action"]
mouseClicksPre.head()

11573  clicks


,username,actionId,localTimestamp,serverTimestamp,action
0,101BSCE120003,ksPnwtEMzNFyjDoPb,1488796295459,1488796295703,Click
1,101BSCE120003,WwR8e66RKxWB8ab7e,1488796296027,1488796296249,Click
2,101BSCE120003,srCWodk9K6YsrxTML,1488796296403,1488796296650,Click
3,101BSCE120003,8TsGqXjMRxCDZ47wQ,1488796298115,1488796298422,Click
4,101BSCE120003,js36dPcccXmZH4fp7,1488796355318,1488796355445,Click


In [111]:
scrollsPre = pd.read_csv("Tablas generadas/Pre-Test/ScrollMoves.PreTest.csv")
scrollsPre = scrollsPre[["username","X_id","localTimestamp","serverTimestamp"]]
actions = np.repeat('Scroll',len(scrollsPre))
scrollsPre['action'] = actions
scrollsPre.columns = ["username","actionId","localTimestamp","serverTimestamp","action"]
print(len(scrollsPre), "scrolls")
scrollsPre.head()

78080 scrolls


,username,actionId,localTimestamp,serverTimestamp,action
0,101BSCE120003,8hMHWgfKp3XAifx7m,1488796059472,1488796059765,Scroll
1,101BSCE120003,GTmc6XXe4JHkvyM3S,1488796064039,1488796064204,Scroll
2,101BSCE120003,s6uWaQxpTjzFMFzGj,1488796065388,1488796065551,Scroll
3,101BSCE120003,Saw37kgjSe3XD6sBs,1488796104153,1488796104456,Scroll
4,101BSCE120003,NCeRGGvzrXayjjd6G,1488796192735,1488796192980,Scroll


En el caso de la entrada de texto se contabilizan cuando se presionan flechas para detectar actividad del usuario en la página.

In [112]:
keystrokesPre = pd.read_csv("Tablas generadas/Pre-test/Keystrokes.PreTest.csv")
print(len(keystrokesPre), " keystrokes")
arrowPressPre = keystrokesPre[(keystrokesPre["keyCode"] == 38) | (keystrokesPre["keyCode"] == 40)]
arrowPressPre = arrowPressPre[["username","userId","localTimestamp","serverTimestamp"]]
actions = np.repeat('ArrowKey',len(arrowPressPre))
arrowPressPre['action'] = actions
arrowPressPre.columns = ["username","actionId","localTimestamp","serverTimestamp","action"]
print(len(arrowPressPre)," arrow press")
arrowPressPre.head()

1135698  keystrokes
4010  arrow press


,username,actionId,localTimestamp,serverTimestamp,action
373,101BSCE120003,KnqPytrKdYvoWobR6,1.488797e+12,1488797082493,ArrowKey
34100,101BSSA110011,gLDBxT8bE4hQHJwN3,1.487848e+12,1487848109770,ArrowKey
36089,101BSSA110011,gLDBxT8bE4hQHJwN3,1.487849e+12,1487849006600,ArrowKey
57581,102BSCE120003,YEtPcxMzxZWganuJf,1.487588e+12,1487587807180,ArrowKey
78087,102BSCE120017,LqLxDjphjYvhu9pPA,1.487588e+12,1487588019105,ArrowKey


Unir elementos en una tabla

In [126]:
actionsInSearchTask = pd.concat([eventlogsPre,mouseClicksPre,scrollsPre,arrowPressPre])

Obtener conjunto de elementos que ocurren en la etapa de búsqueda y selección

In [161]:
def getEventsInSearchTask (users,events,timeTable):
    columns = ['username','actionId','localTimestamp','serverTimestamp','action']
    acumulator = pd.DataFrame(columns=columns)
    for i in range(len(users["userName"])):
        actionsPerUser = events[
            (events['serverTimestamp'] > timeTable['start'][i]) &
            (events['serverTimestamp'] < timeTable['finish'][i]) &
            (events['username'] == users['userName'][i])
        ]
        acumulator = pd.concat([acumulator,actionsPerUser])
    return acumulator

In [168]:
actionsInSearchTask = getEventsInSearchTask(usersPre,eventlogsPre,timeTablePre)
actionsInSearchTask.head()

,username,actionId,localTimestamp,serverTimestamp,action
46,101BSCE120003,8GExxyQeMScDeQgbB,1488795953616,1488795953964,PageEnter
47,101BSCE120003,BAmeJRb6f9Lrzuuoo,1488795990019,1488795990319,Query
48,101BSCE120003,NaN,1488796000582,1488796000582,StatusAway
49,101BSCE120003,rGaDb5iAEwqngG7Li,1488796005341,1488796005639,Query
50,101BSCE120003,NaN,1488796006288,1488796006289,StatusAway


Aún existen algunos datos que no serán de utilidad para la investigación del comportamiento del usuario. Estos son removidos.

In [171]:
actionsInSearchTask = actionsInSearchTask[
    (actionsInSearchTask["action"]!="StatusAway") & 
    (actionsInSearchTask["action"]!="StatusOnline") &
    (actionsInSearchTask["action"]!="BookmarkSelected") &
    (actionsInSearchTask["action"]!="StatusOffline") &
    (actionsInSearchTask["action"]!="TutorialSelected") &
    (actionsInSearchTask["action"]!="FormResponse") &
    (actionsInSearchTask["action"]!="Login") &
    (actionsInSearchTask["action"]!="SubtaskSelected") &
    (actionsInSearchTask["action"]!="Logout") &
    (actionsInSearchTask["action"]!="TimeoutTriggered")
                                         ]
actionsInSearchTask.head(10)

,username,actionId,localTimestamp,serverTimestamp,action
46,101BSCE120003,8GExxyQeMScDeQgbB,1488795953616,1488795953964,PageEnter
47,101BSCE120003,BAmeJRb6f9Lrzuuoo,1488795990019,1488795990319,Query
49,101BSCE120003,rGaDb5iAEwqngG7Li,1488796005341,1488796005639,Query
53,101BSCE120003,AXEv4fGMmYkmhebxu,1488796037651,1488796037940,Query
54,101BSCE120003,FajRFTo5JeMzJWhca,1488796047341,1488796047520,Query
55,101BSCE120003,NaN,1488796051784,1488796051949,SearchResultSelected
56,101BSCE120003,vNw9Suw42CF2hq9gN,1488796051793,1488796051953,PageExit
57,101BSCE120003,edkYh8cwLdzmN8nmr,1488796051795,1488796051964,PageEnter
60,101BSCE120003,83hA48zwoGzRJHmQj,1488796108619,1488796110289,Bookmark
61,101BSCE120003,NaN,1488796113250,1488796113513,BackButtonSelected
